# Building Multimodal AI Applications with LangChain & the OpenAI API 

## Goals 

Videos can be full of useful information, but getting hold of that info can be slow, since you need to watch the whole thing or try skipping through it. It can be much faster to use a bot to ask questions about the contents of the transcript.

In this project, you'll download a tutorial video from YouTube, transcribe the audio, and create a simple Q&A bot to ask questions about the content.

- Understanding the building blocks of working with Multimodal AI projects
- Working with some of the fundamental concepts of LangChain  
- How to use the Whisper API to transcribe audio to text 
- How to combine both LangChain and Whisper API to create ask questions of any YouTube video 

## Before you begin

You'll need a developer account with [OpenAI ](https://auth0.openai.com/u/signup/identifier?state=hKFo2SAyeTZBU1pzbUNWYWs3Wml5OWVvUVh4enZldC1LYU9PMaFur3VuaXZlcnNhbC1sb2dpbqN0aWTZIDFUakNoUGFMLUdNWFpfQkpqdncyZjVDQk9xUTE4U0xDo2NpZNkgRFJpdnNubTJNdTQyVDNLT3BxZHR3QjNOWXZpSFl6d0Q) and a create API Key. The API secret key will be stored in your 'Environment Variables' on the side menu. See the *getting-started.ipynb* notebook for details on setting this up.

## Task 0: Setup

The project requires several packages that need to be installed into Workspace.

- `langchain` is a framework for developing generative AI applications.
- `yt_dlp` lets you download YouTube videos.
- `tiktoken` converts text into tokens.
- `docarray` makes it easier to work with multi-model data (in this case mixing audio and text).

### Instructions

Run the following code to install the packages.

In [ ]:
# Lock OpenAI version to 0.27.1
!pip install openai==0.27.1
# Install langchain
!pip install langchain==0.0.292

In [ ]:
# Install yt_dlp
!pip install yt_dlp==2023.7.6

In [ ]:
!pip install tiktoken==0.5.1

In [ ]:
!pip install docarray==0.38.0

### Instructions

## Task 1: Import The Required Libraries 

For this project we need the `os` and the `yt_dlp` packages to download the YouTube video of your choosing, convert it to an `.mp3` and save the file. We will also be using the `openai` package to make easy calls to the OpenAI models we will use. 

Import the following packages.

- Import `os` 
- Import `openai`
- Import `yt_dlp` with the alias `youtube_dl`
- From the `yt_dlp` package, import `DowloadError`
- Assign `openai_api_key` to `os.getenv("OPENAI_API_KEY")`

In [7]:
pip install --quiet openai yt_dlp docarray


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
# Importing the Required Packages including: "os" "openai" "yt_dlp as youtube_dl" and "from yt_dl import Download Error"

# Import the os package
import os 

# Import the glob package
import glob

# Import the openai package 
import openai 

# Import the yt_dlp package as youtube_dl
import yt_dlp as youtube_dl

# Import DownloadError from yt_dlp
from yt_dlp import DownloadError 

# Import DocArray 
import docarray 


We will also assign the variable `openai_api_key` to the environment variable "OPEN_AI_KEY". This will help keep our key secure and remove the need to write it in the code here. 

In [9]:
openai_api_key = os.getenv("OPENAI_API_KEY")

## Task 2: Download the YouTube Video

After creating the setup, the first step we will need to do is download the video from Youtube and convert it to an audio file (.mp3). 

We'll download a DataCamp tutorial about machine learning in Python.

We will do this by setting a variable to store the `youtube_url` and the `output_dir` that we want the file to be stored. 

The `yt_dlp` allows us to download and convert in a few steps but does require a few configuration steps. This code is provided to you. 

Lastly, we will create a loop that looks in the `output_dir` to find any .mp3 files. Then we will store those in a list called `audio_files` that will be used later to send each file to the Whisper model for transcription. 

Create the following: 
- Two variables - `youtube_url` to store the Video URL and `output_dir` that will be the directory where the audio files will be saved. 
- For this tutorial, we can set the `youtube_url` to the following `"https://www.youtube.com/watch?v=aqzxYofJ_ck"`and the `output_dir`to `files/audio/`. In the future, you can change these values. 
- Use the `ydl_config` that is provided to you 

In [10]:
# An example YouTube tutorial video
youtube_url = "https://www.youtube.com/watch?v=aqzxYofJ_ck"
# Directory to store the downloaded video
output_dir = "files/audio/"

# Config for youtube-dl
ydl_config = {
    "format": "bestaudio/best",
    "postprocessors": [
        {
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",
            "preferredquality": "192",
        }
    ],
    "outtmpl": os.path.join(output_dir, "%(title)s.%(ext)s"),
    "verbose": True
}

# Check if the output directory exists, if not create it
if not os.path.exists(output_dir): 
    os.makedirs(output_dir)


# Print a message indicating which video is being downloaded

print(f"Downloading video from {youtube_url}")


# Attempt to download the video using the specified configuration
# If a DownloadError occurs, attempt to download the video again

try: 
    with youtube_dl.YoutubeDL(ydl_config) as ydl: 
        ydl.download([youtube_url])
except DownloadError: 
    with youtube_dl.YoutubeDL(ydl_config) as ydl: 
        ydl.download([youtube_url])



[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out UTF-8 (No ANSI), error UTF-8 (No ANSI), screen UTF-8 (No ANSI)
[debug] yt-dlp version stable@2024.05.27 from yt-dlp/yt-dlp [12b248ce6] (pip) API
[debug] params: {'format': 'bestaudio/best', 'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'mp3', 'preferredquality': '192'}], 'outtmpl': 'files/audio/%(title)s.%(ext)s', 'verbose': True, 'compat_opts': set(), 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Language': 'en-us,en;q=0.5', 'Sec-Fetch-Mode': 'navigate'}}
[debug] Python 3.10.14 (CPython aarch64 64bit) - Linux-6.6.22-linuxkit-aarch64-with-glibc2.36 (OpenSSL 3.0.11 19 Sep 2023, glibc 2.36)
[debug] exe versions: none
[debug] Optional libraries: Cryptodome-3.20.0, brotli-1.1.0, certifi-2024.06.02, mutagen-1.47.0, requests-2.3

[youtube] Extracting URL: https://www.youtube.com/watch?v=aqzxYofJ_ck
[youtube] aqzxYofJ_ck: Downloading webpage
[youtube] aqzxYofJ_ck: Downloading ios player API JSON
[youtube] aqzxYofJ_ck: Downloading player 61b3b5e4


[debug] Saving youtube-nsig.61b3b5e4 to cache
[debug] [youtube] Decrypted nsig SKp_xmEhODLjoMbpWTr => nXwxZc92c5r72g
[debug] Loading youtube-nsig.61b3b5e4 from cache
[debug] [youtube] Decrypted nsig 6b5FzQE-7pHjYUiCGcE => j6ykdznaMMN4IA


[youtube] aqzxYofJ_ck: Downloading m3u8 information


[debug] Sort order given by extractor: quality, res, fps, hdr:12, source, vcodec:vp9.2, channels, acodec, lang, proto
[debug] Formats sorted by: hasvid, ie_pref, quality, res, fps, hdr:12(7), source, vcodec:vp9.2(10), channels, acodec, lang, proto, size, br, asr, vext, aext, hasaud, id


...

[info] aqzxYofJ_ck: Downloading 1 format(s): 251


[debug] Invoking http downloader on "https://rr3---sn-tt1e7nlz.googlevideo.com/videoplayback?expire=1719360903&ei=Jwl7ZquxIIeK2_gPt9uz8Ao&ip=209.120.255.181&id=o-AA57QPETmNOpYdDlKg8W7SrURt9g6IGaYdQTaS9uqAMY&itag=251&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&mh=zw&mm=31%2C26&mn=sn-tt1e7nlz%2Csn-vgqsrnzy&ms=au%2Conr&mv=m&mvi=3&pl=24&initcwndbps=903750&bui=AbKP-1PWx6IRSpnhl6k575RkYmnUwDvN1EuSdnN-z6dhU6O8iCXCFTxg7WrQUYA92Nux4pJ76kNyNY9E&spc=UWF9fyAxNUn4JDEoA3km2bbYb0sf1Zizx4NpOy5U9ACCLiFLPNzybwJ9Un7O&vprv=1&svpuc=1&mime=audio%2Fwebm&ns=Mx8MMVlIl7lys9j2-xrG4F8Q&rqh=1&gir=yes&clen=10932652&dur=752.701&lmt=1654008313150389&mt=1719338950&fvip=3&keepalive=yes&c=WEB&sefc=1&txp=5318224&n=yqoKHCw0bCY_7w&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cxpc%2Cbui%2Cspc%2Cvprv%2Csvpuc%2Cmime%2Cns%2Crqh%2Cgir%2Cclen%2Cdur%2Clmt&sig=AJfQdSswRgIhAMF5qg0HRJUUMCr2_RkNJnvSf6q8GxXUyOkHGvUI4mUGAiEAw5APq8ZnXkvLM_8lOsgQx9O5XbQY5LIVhylLf87TjDw%3D&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2

[download] files/audio/Python Machine Learning Tutorial ｜ Splitting Your Data ｜ Databytes.webm has already been downloaded
[download] 100% of   10.43MiB


ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location
Traceback (most recent call last):
  File "/home/vscode/.local/lib/python3.10/site-packages/yt_dlp/YoutubeDL.py", line 3537, in process_info
    replace_info_dict(self.post_process(dl_filename, info_dict, files_to_move))
  File "/home/vscode/.local/lib/python3.10/site-packages/yt_dlp/YoutubeDL.py", line 3719, in post_process
    info = self.run_all_pps('post_process', info, additional_pps=info.get('__postprocessors'))
  File "/home/vscode/.local/lib/python3.10/site-packages/yt_dlp/YoutubeDL.py", line 3701, in run_all_pps
    info = self.run_pp(pp, info)
  File "/home/vscode/.local/lib/python3.10/site-packages/yt_dlp/YoutubeDL.py", line 3679, in run_pp
    files_to_delete, infodict = pp.run(infodict)
  File "/home/vscode/.local/lib/python3.10/site-packages/yt_dlp/postprocessor/common.py", line 23, in run
    ret = func(self, info, *args, **kwargs)
  File "/home/vscode/.local/l

DownloadError: ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location

To find the audio files that we will use the `glob`module that looks in the `output_dir` to find any .mp3 files. Then we will append the file to a list called `audio_files`. This will be used later to send each file to the Whisper model for transcription. 

Create the following: 
- A variable called `audio_files`that uses the glob module to find all matching files with the `.mp3` file extension 
- Select the first first file in the list and assign it to `audio_filename`
- To verify the filename, print `audio_filename` 

In [ ]:
# Find the audio file in the output directory

# Find all the audio files in the output directory
audio_files = glob.glob(os.path.join(output_dir, "*.mp3"))


# Select the first audio file in the list
audio_filename = audio_files[0]

# Print the name of the selected audio file
print(audio_filename)

## Task 3: Transcribe the Video using Whisper

In this step we will take the downloaded and converted Youtube video and send it to the Whisper model to be transcribed. To do this we will create variables for the `audio_file`, for the `output_file` and the model. 

Using these variables we will:
- create a list to store the transcripts
- Read the Audio File 
- Send the file to the Whisper Model using the OpenAI package 

To complete this step, create the following: 
- A variable named `audio_file`that is assigned the `audio_filename` we created in the last step
- A variable named `output_file` that is assigned the value `"files/transcripts/transcript.txt"`
- A variable named `model` that is assigned the value  `"whisper-1"`
- An empty list called `transcripts`
- A variable named `audio` that uses the `open` method and `"rb"` modifier on the `audio_file`
- A variable to store the `response` from the `openai.Audio.transcribe` method that takes in the `model`and `audio` variables 
- Append the `response["text"]`to the `transcripts` list. 

In [ ]:
# Define function parameters
audio_file = audio_filename
output_file = "files/transcripts/transcript.txt"
model = "whisper-1"

# Print the name of the audio file
print(audio_file)


# Transcribe the audio file to text using OpenAI API
print("converting audio to text...")

with open(audio_file, "rb") as audio:
    response = openai.Audio.transcribe(model, audio)

# Extract the transcript from the response
transcript = (response["text"])

To save the transcripts to text files we will use the below provided code: 

In [ ]:
# If an output file is specified, save the transcript to a .txt file

if output_file is not None:
    # Create the directory for the output file if it doesn't exist
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    # Write the transcript to the output file
    with open(output_file, "w") as file:
        file.write(transcript)

# Print the transcript to the console to verify it worked 
print(transcript)



## Task 4: Create a TextLoader using LangChain 

In order to use text or other types of data with LangChain we must first convert that data into Documents. This is done by using loaders. In this tutorial, we will use the `TextLoader` that will take the text from our transcript and load it into a document. 

To complete this step, do the following: 
- Import `TextLoader` from `langchain.document_loaders`
- Create a variable called `loader` that uses the `TextLoader` method which takes in the directory of the transcripts `"./files/text"`
- Create a variable called `docs` that is assigned the result of calling the `loader.load()` method. 

In [ ]:
# Import the TextLoader class from the langchain.document_loaders module

from langchain.document_loaders import TextLoader

# Create a new instance of the TextLoader class, specifying the directory containing the text files

loader = TextLoader("./files/text")

# Load the documents from the specified directory using the TextLoader instance

docs = loader.load()

In [ ]:
# Show the first element of docs to verify it has been loaded 
docs[0]

## Task 4: Creating an In-Memory Vector Store 

Now that we have created Documents of the transcription, we will store that Document in a vector store. Vector stores allows LLMs to traverse through data to find similiarity between different data based on their distance in space. 

For large amounts of data, it is best to use a designated Vector Database. Since we are only using one transcript for this tutorial, we can create an in-memory vector store using the `docarray` package. 

We will also tokenize our queries using the `tiktoken` package. This means that our query will be seperated into smaller parts either by phrases, words or characters. Each of these parts are assigned a token which helps the model "understand" the text and relationships with other tokens. 

### Instructions

- Import the `tiktoken` package. 

In [ ]:
# Import the tiktoken package
import tiktoken

## Task 5: Create the Document Search 

We will now use LangChain to complete some important operations to create the Question and Answer experience. Let's import the following: 

- Import `RetrievalQA` from `langchain.chains` - this chain first retrieves documents from an assigned Retriver and then runs a QA chain for answering over those documents 
- Import `ChatOpenAI` from `langchain.chat_models` - this imports the ChatOpenAI model that we will use to query the data 
- Import `DocArrayInMemorySearch` from `langchain.vectorstores` - this gives the ability to search over the vector store we have created. 
- Import `OpenAIEmbeddings` from `langchain.embeddings` - this will create embeddings for the data store in the vector store. 
- Import `display` and `Markdown`from `IPython.display` - this will create formatted responses to the queries. (

In [ ]:
# Import the RetrievalQA class from the langchain.chains module
from langchain.chains import RetrievalQA

# Import the ChatOpenAI class from the langchain.chat_models module
from langchain.chat_models import ChatOpenAI

# Import the DocArrayInMemorySearch class from the langchain.vectorstores module
from langchain.vectorstores import DocArrayInMemorySearch

# Import the OpenAIEmbeddings class from the langchain.embeddings module
from langchain.embeddings import OpenAIEmbeddings

Now we will create a vector store that will use the `DocArrayInMemory` search methods which will search through the created embeddings created by the OpenAI Embeddings function. 

To complete this step: 
- Create a variable called `db`
- Assign the `db` variable to store the result of the method `DocArrayInMemorySearch.from_documents`
- In the DocArrayInMemorySearch method, pass in the `docs` and a function call to `OpenAIEmbeddings()`

In [ ]:
# Create a new DocArrayInMemorySearch instance from the specified documents and embeddings
db = DocArrayInMemorySearch.from_documents(
    docs, 
    OpenAIEmbeddings()
)

We will now create a retriever from the `db` we created in the last step. This enables the retrieval of the stored embeddings. Since we are also using the `ChatOpenAI` model, will assigned that as our LLM.

Create the following: 
- A variable called `retriever` that is assigned `db.as_retriever()`
- A variable called `llm` that creates the `ChatOpenAI` method with a set `temperature`of `0.0`. This will controle the variability in the responses we receive from the LLM. 

In [ ]:
# Convert the DocArrayInMemorySearch instance to a retriever
retriever = db.as_retriever()

# Create a new ChatOpenAI instance with a temperature of 0.0
llm = ChatOpenAI(temperature = 0.0)

Our last step before starting to ask questions is to create the `RetrievalQA` chain. This chain takes in the:  
- The `llm` we want to use
- The `chain_type` which is how the model retrieves the data
- The `retriever` that we have created 
- An option called `verbose` that allows use to see the seperate steps of the chain 

Create a variable called `qa_stuff`. This variable will be assigned the method `RetrievalQA.from_chain_type`. 

Use the following settings inside this method: 
- `llm=llm`
- `chain_type="stuff"`
- `retriever=retriever`
- `verbose=True`

In [ ]:
# Create a new RetrievalQA instance with the specified parameters
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,            # The ChatOpenAI instance to use for generating responses
    chain_type="stuff", # The type of chain to use for the QA system
    retriever=retriever, # The retriever to use for retrieving relevant documents
    verbose=True        # Whether to print verbose output during retrieval and generation
)

## Task 5: Create the Queries 

Now we are ready to create queries about the YouTube video and read the responses from the LLM. This done first by creating a query and then running the RetrievalQA we setup in the last step and passing it the query. 

To create the questions to ask the model complete the following steps: 
- Create a variable call `query` and assigned it a string value of `"What is this tutorial about?"`
- Create a `response` variable that will store the result of `qa_stuff.run(query)` 
- Show the `resposnse`

In [ ]:
# Set the query to be used for the QA system
query = "What is this tutorial about?"

# Run the query through the RetrievalQA instance and store the response
response = qa_stuff.run(query)

# Print the response to the console
response


We can continue on creating queries and even creating queries that we know would not be answered in this video to see how the model responds. 

In [ ]:
# Set the query to be used for the QA system
query = "What is the difference between a training set and test set?"

# Run the query through the RetrievalQA instance and store the response
response = qa_stuff.run(query)

# Print the response to the console
response

In [ ]:
# Set the query to be used for the QA system
query = "Who should watch this lesson?"

# Run the query through the RetrievalQA instance and store the response
response = qa_stuff.run(query)

# Print the response to the console
response 

In [ ]:
# Set the query to be used for the QA system
query ="Who is the greatest football team on earth?"

# Run the query through the RetrievalQA instance and store the response
response = qa_stuff.run(query)

# Print the response to the console
response 

In [ ]:
# Set the query to be used for the QA system
query = "How long is the circumference of the earth?"

# Run the query through the RetrievalQA instance and store the response
response = qa_stuff.run(query)

# Print the response to the console
response 

## All done, congrats! 